In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Mon Sep 13 07:20:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os

os.getcwd()

'/content'

In [ ]:
os.chdir("/content/drive/MyDrive/Colab Notebooks/qbe/candidate_test")

In [ ]:
os.listdir()

['restaurants_train.csv',
 'restaurants_val.csv',
 'requirements.txt',
 'restaurants_holdout.csv',
 'README.pdf',
 '.~lock.restaurants_train.csv#',
 'entity_extraction.ipynb',
 '.ipynb_checkpoints',
 'rest_hold_preds_batchsize10.xlsx',
 'rest_hold_preds_batchsize10_20epochs.xlsx',
 'b_10_ss5',
 'b_10_cosine_sched']

In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 2.5 MB 8.0 MB/s 
     |████████████████████████████████| 804.1 MB 2.8 kB/s 
     |████████████████████████████████| 10.1 MB 38.6 MB/s 
     |████████████████████████████████| 3.3 MB 45.9 MB/s 
     |████████████████████████████████| 895 kB 42.4 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu102 requires torch==1.9.0, but you have torch 1.8.1 which is incompatible.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.8.1 which is incompatible.
google-colab 1.0.0 requires pand

In [ ]:
import pandas as pd

train = pd.read_csv('restaurants_train.csv')

val = pd.read_csv('restaurants_val.csv')

holdout = pd.read_csv('restaurants_holdout.csv')

train = train.fillna('')

val = val.fillna('')

holdout = holdout.fillna('')

In [ ]:
import sys
import torch
import torch.nn as nn
import random

from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR
from transformers import BartForConditionalGeneration, BartTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import transformers

In [ ]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

_ = model.to(device)

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [ ]:
def generate_outputs(model, sentence):
    inputs = tokenizer([sentence], max_length=1024, return_tensors='pt')
    output_ids = model.generate(inputs['input_ids'].to(device), num_beams=4)
    #print(output_ids, sentence)
    output = tokenizer.decode(output_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False).strip()
    return output

In [ ]:
train

In [ ]:
val

In [ ]:
model.eval()
generate_outputs(model, train.sentence[6])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


'can you tell me where the nearest wendys is'

In [ ]:
model.train()
1

1

In [ ]:
################################
# DO NOT CHANGE THIS FUNCTION! #
################################

def get_f1_score_on_test_data(model, data):
    model.eval()
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    for index, row in data.iterrows():
        sentence = row.sentence
        expected = row.restaurant_name
        inputs = tokenizer([sentence], max_length=1024, return_tensors='pt')
        predicted = generate_outputs(model, sentence)
        if expected != '' and expected == predicted:
            true_positives += 1
        if expected != '' and expected != predicted:
            false_positives += 1
        if expected == '' and predicted != '':
            false_positives += 1
        if expected != '' and predicted == '':
            false_negatives += 1

    precision = 0
    recall = 0
    f1_score = 0
    if true_positives + false_positives:
        precision = true_positives / (true_positives + false_positives)
    if true_positives + false_negatives:
        recall = true_positives /(true_positives + false_negatives)
    if precision + recall:
        f1_score = 2 * precision * recall / (precision + recall)

    print(f'precision: {precision} | recall {recall} | f1_score {f1_score}')
    return f1_score

In [ ]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

_ = model.to(device)

num_epochs = 20
learning_rate = 2e-5
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#scheduler = StepLR(optimizer, step_size=10, gamma=0.8)

In [ ]:
# from torch.nn.utils.rnn import pad_sequence

In [ ]:
# tokenizer.batch_encode_plus?

In [ ]:
# train.sentence[0]

In [ ]:
# tokenizer.batch_encode_plus([train.sentence[0]])["input_ids"]

In [ ]:
# tokenizer.encode?
# #(train.sentence[0], return_tensors='pt')

In [ ]:
# sents = [x.sentence for row, x in train.iterrows()]

# labels = [x.restaurant_name for row, x in train.iterrows()]

# start = 0

# tokenizer.batch_encode_plus(sents[start:start+10], padding=True)["input_ids"]

In [ ]:
def train_one_batch_and_get_loss(train_model, data, optimizer, criterion, batch_size=10):
    total_loss = 0.

    sents = [x.sentence for row, x in data.iterrows()]
    labels = [x.restaurant_name for row, x in data.iterrows()]

    #sents_tok = [tokenizer.encode(x, padding=True) for x in sents]

    #labels_tok = [tokenizer.encode(x, padding=True) for x in labels]

    total_steps = int(len(sents) / batch_size)

    for i in tqdm(range(total_steps), total=total_steps): 
        start = i * batch_size       
        #print(start, start+batch_size)
        batch_sents = torch.tensor(tokenizer.batch_encode_plus(sents[start:start+batch_size], padding=True)["input_ids"]).to(device)
        
        batch_labels = torch.tensor(tokenizer.batch_encode_plus(labels[start:start+batch_size], padding=True)["input_ids"]).to(device)
        model.train()
        optimizer.zero_grad()
        
        loss = train_model(batch_sents, labels=batch_labels)[0]
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / data.shape[0]

In [ ]:
def train_one_step_and_get_loss(train_model, data, optimizer, criterion):
    total_loss = 0.
    for i, row in tqdm(data.iterrows(), total=data.shape[0]):
        model.train()
        optimizer.zero_grad()
        input_ids = tokenizer.encode(row.sentence, return_tensors='pt').to(device)
        output_ids = tokenizer.encode(row.restaurant_name, return_tensors='pt').to(device)
        loss = train_model(input_ids, labels=output_ids)[0]
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / data.shape[0]

In [ ]:
best_model = None
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

_ = model.to(device)

num_epochs = 100
learning_rate = 2e-5
batch_size = 10
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#scheduler = StepLR(optimizer, step_size=10, gamma=0.8)

num_total_steps = int(train.shape[0] * num_epochs / (batch_size))
num_warmup_steps = int(0.05 * num_total_steps)
scheduler = transformers.get_cosine_schedule_with_warmup(optimizer,num_warmup_steps=num_warmup_steps, 
                                                         num_training_steps=num_total_steps)
min_f1 = 0
holdout_f1 = 0
MODEL_PATH = "b_10_cosine_sched"
for epoch in range(num_epochs):
    loss = train_one_batch_and_get_loss(model, train, optimizer, criterion, batch_size=batch_size)
    print('Epoch:', epoch , '-' * 35)
    print('Training loss:', loss)
    f1_val = get_f1_score_on_test_data(model, val)
    
    if (f1_val >= min_f1) & (f1_val > 0):

        f1 = get_f1_score_on_test_data(model, holdout)        
        print("saving model:", f1, f1_val,min_f1, epoch)
        min_f1 = f1_val
        holdout_f1 = f1
        torch.save(model.state_dict(), MODEL_PATH)
        f1_train = get_f1_score_on_test_data(model, train)
        print("Train f1", f1_train)
    if epoch % 5 == 0:
        print("holdout eval")
        f1 = get_f1_score_on_test_data(model, holdout)
    best_model = model
    print('-' * 44)
    sys.stdout.flush()
    scheduler.step()

100%|██████████| 12/12 [00:02<00:00,  4.48it/s]
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Epoch: 0 -----------------------------------
Training loss: 1.0764890273412069
precision: 0.0 | recall 0 | f1_score 0
holdout eval
precision: 0.0 | recall 0 | f1_score 0
--------------------------------------------


100%|██████████| 12/12 [00:02<00:00,  4.77it/s]


Epoch: 1 -----------------------------------
Training loss: 1.0803970495859783
precision: 0.0 | recall 0 | f1_score 0
--------------------------------------------


100%|██████████| 12/12 [00:02<00:00,  4.75it/s]


...

Epoch: 97 -----------------------------------
Training loss: 0.0022356394367913407
precision: 0.6666666666666666 | recall 0.8888888888888888 | f1_score 0.761904761904762
precision: 0.30275229357798167 | recall 0.868421052631579 | f1_score 0.4489795918367347
saving model: 0.4489795918367347 0.761904761904762 0.761904761904762 97
precision: 0.9069767441860465 | recall 1.0 | f1_score 0.951219512195122
Train f1 0.951219512195122
--------------------------------------------


100%|██████████| 12/12 [00:02<00:00,  4.56it/s]


Epoch: 98 -----------------------------------
Training loss: 0.0030227651353925467
precision: 0.6666666666666666 | recall 0.8888888888888888 | f1_score 0.761904761904762
precision: 0.2815533980582524 | recall 0.7837837837837838 | f1_score 0.4142857142857143
saving model: 0.4142857142857143 0.761904761904762 0.761904761904762 98
precision: 0.9069767441860465 | recall 1.0 | f1_score 0.951219512195122
Train f1 0.951219512195122
--------------------------------------------


100%|██████████| 12/12 [00:02<00:00,  4.55it/s]


Epoch: 99 -----------------------------------
Training loss: 0.0007436891008789341
precision: 0.6666666666666666 | recall 0.8888888888888888 | f1_score 0.761904761904762
precision: 0.27884615384615385 | recall 0.7631578947368421 | f1_score 0.4084507042253521
saving model: 0.4084507042253521 0.761904761904762 0.761904761904762 99
precision: 0.9069767441860465 | recall 1.0 | f1_score 0.951219512195122
Train f1 0.951219512195122
--------------------------------------------


In [ ]:
holdout_f1

0.40268456375838924

In [ ]:
best_model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
best_model.load_state_dict(torch.load(MODEL_PATH))
best_model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        

In [ ]:
get_f1_score_on_test_data(best_model, holdout)

precision: 0.27884615384615385 | recall 0.7631578947368421 | f1_score 0.4084507042253521


0.4084507042253521

In [ ]:
get_f1_score_on_test_data(model, holdout)

precision: 0.27884615384615385 | recall 0.7631578947368421 | f1_score 0.4084507042253521


0.4084507042253521

In [ ]:
get_f1_score_on_test_data(model, train)

precision: 0.8604651162790697 | recall 1.0 | f1_score 0.9249999999999999


0.9249999999999999

In [ ]:
best_model = None
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

_ = model.to(device)

num_epochs = 20
learning_rate = 2e-6
batch_size = 10
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#scheduler = StepLR(optimizer, step_size=10, gamma=0.8)

num_total_steps = int(train.shape[0] * num_epochs / (batch_size))
num_warmup_steps = int(0.1 * num_total_steps)
scheduler = transformers.get_cosine_schedule_with_warmup(optimizer,num_warmup_steps=num_warmup_steps, 
                                                         num_training_steps=num_total_steps)
min_f1 = 0
holdout_f1 = 0
MODEL_PATH = "b_10_cosine_sched"
for epoch in range(num_epochs):
    loss = train_one_batch_and_get_loss(model, train, optimizer, criterion, batch_size=batch_size)
    print('Epoch:', epoch , '-' * 35)
    print('Training loss:', loss)
    f1_val = get_f1_score_on_test_data(model, val)
    
    if (f1_val >= min_f1) & (f1_val > 0):

        f1 = get_f1_score_on_test_data(model, holdout)        
        print("saving model:", f1, f1_val,min_f1, epoch)
        min_f1 = f1_val
        holdout_f1 = f1
        torch.save(model.state_dict(), MODEL_PATH)
        f1_train = get_f1_score_on_test_data(model, train)
        print("Train f1", f1_train)
    if epoch % 5 == 0:
        print("holdout eval")
        f1 = get_f1_score_on_test_data(model, holdout)
    best_model = model
    print('-' * 44)
    sys.stdout.flush()
    scheduler.step()

100%|██████████| 12/12 [00:02<00:00,  4.62it/s]
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Epoch: 0 -----------------------------------
Training loss: 1.0977604866027832
precision: 0.0 | recall 0 | f1_score 0
holdout eval
precision: 0.0 | recall 0 | f1_score 0
--------------------------------------------


100%|██████████| 12/12 [00:02<00:00,  4.66it/s]


Epoch: 1 -----------------------------------
Training loss: 1.0879191716512044
precision: 0.0 | recall 0 | f1_score 0
--------------------------------------------


100%|██████████| 12/12 [00:02<00:00,  4.66it/s]


...

Epoch: 17 -----------------------------------
Training loss: 0.7906840761502584
precision: 0.0 | recall 0.0 | f1_score 0
--------------------------------------------


100%|██████████| 12/12 [00:02<00:00,  4.67it/s]


Epoch: 18 -----------------------------------
Training loss: 0.7845533808072408
precision: 0.0 | recall 0.0 | f1_score 0
--------------------------------------------


100%|██████████| 12/12 [00:02<00:00,  4.68it/s]


Epoch: 19 -----------------------------------
Training loss: 0.7756455818812052
precision: 0.0 | recall 0.0 | f1_score 0
--------------------------------------------


In [ ]:
model.eval()
holdout["preds"] = holdout["sentence"].apply(lambda x: generate_outputs(model, x))

holdout

,sentence,restaurant_name,preds
0,find pizza places,,
1,find me the best rated chinese restaurant in t...,,chinese restaurant
2,what kind of food does abc cafe serve,abc cafe,abc cafe
3,how far away is the nearest steak house,,
4,i am looking for a mexican restuarant that has...,,
...,...,...,...
145,find me brazilian food with on location parking,,
146,get me to a mexican place,,
147,how far am i from the nearest bagel shop,,bagel shop
148,what time does sonic open,sonic,


In [ ]:
def generate_outputs(model, sentence):
    inputs = tokenizer([sentence], max_length=1024, return_tensors='pt')
    output_ids = model.generate(inputs['input_ids'].to(device), num_beams=4)
    #print(output_ids, sentence)
    output = tokenizer.decode(output_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False).strip()

    if output in sentence:
        return output
    else:
        return ""

In [ ]:
get_f1_score_on_test_data(model, holdout)

precision: 0.30337078651685395 | recall 0.6136363636363636 | f1_score 0.406015037593985


0.406015037593985

In [ ]:
holdout["preds2"] = holdout["sentence"].apply(lambda x: generate_outputs(model, x))

holdout

,sentence,restaurant_name,preds,preds2
0,find pizza places,,,
1,find me the best rated chinese restaurant in t...,,chinese restaurant,chinese restaurant
2,what kind of food does abc cafe serve,abc cafe,abc cafe,abc cafe
3,how far away is the nearest steak house,,,
4,i am looking for a mexican restuarant that has...,,,
...,...,...,...,...
145,find me brazilian food with on location parking,,,
146,get me to a mexican place,,,
147,how far am i from the nearest bagel shop,,bagel shop,bagel shop
148,what time does sonic open,sonic,,


In [ ]:
holdout.to_excel("rest_hold_preds_batchsize10_20epochs.xlsx", index=False)

In [ ]:
num = 0
print('input:', holdout.sentence[num])
print('expected:', holdout.restaurant_name[num])
print('predicted:', generate_outputs(best_model, holdout.sentence[num]))

input: find pizza places
expected: 
predicted: 
